# Import libs

In [8]:
from concurrent import futures
import os,glob,sys,requests
from pathlib import Path


# Download all files

### processed_url.txt :  contain the list of downloaded files 
### failed_url.txt  :    contain the list of files:   failed to download  

In [17]:

save_path="/home/ubuntu/projects/continual/dataset/"
def download_file(url,cls):
    """
    @ url:  url for download
    @ url image class 
    """
    try:
        local_filename = url.split('/')[-1]
        # NOTE the stream=True parameter below
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            save_path_cls=save_path+cls
            print(save_path_cls)
            Path(save_path_cls).mkdir(parents=True, exist_ok=True)
            with open(save_path_cls+"/"+local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192): 
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)
                        # f.flush()
        return [url,cls]
    except:
        return [None,url,cls]

    
with open("vis10cat.txt", "r") as f:
    data=f.readlines()
    
    wait_for = []
    with futures.ThreadPoolExecutor(max_workers=2) as ex:

        for d in data:
            url=d.split("\t")[-1].split("\n")[0]
            cls=d.split("\t")[0]
            w=ex.submit(download_file, url,cls)
            wait_for.append(w)

    f1 = open("processed_url.txt", "a")
    f2 = open("failed_url.txt", "a")

    for f in futures.as_completed(wait_for):
        if f.result()[0]:
            f1.write(f.result()[1]+"\t"+f.result()[0]+"\n")
        else:
            f2.write(f.result()[2]+"\t"+f.result()[1]+"\n")
        print('main: result: {}'.format(f.result()[0]))
    f1.close()
    f2.close()

# Removing currept files and git to png converting

In [ ]:
from PIL import Image
import sys
from pathlib import Path
import cv2,glob,os
import numpy as np
from shutil import copyfile

path_f="/home/ubuntu/projects/continual/dataset/final_clean/"

def processImage(infile):
    """
    @infile : input image path
    @ return :  converted png file path
    """
    try:
        im = Image.open(infile)
    except IOError:
        print( "Cant load", infile)
        return None
    i = 0
    mypalette = im.getpalette()
    if mypalette is None:
        return None
    fname=infile.split("/")[-2]
    save_path_cls=path_f+fname+'/'
    Path(save_path_cls).mkdir(parents=True, exist_ok=True)
    img_name=infile.split(".")[-2].split("/")[-1]+str(i)

    f_save_p=save_path_cls+img_name+'.png'

    try:
        while 1:
            
            im.putpalette(mypalette)
            new_im = Image.new("RGBA", im.size)
            new_im.paste(im)
            new_im.save(f_save_p)

            i += 1
            im.seek(im.tell() + 1)

    except EOFError:
        return f_save_p
        
    return f_save_p


ims=glob.glob("dataset/Table/*")
for img in ims:
    fname=img.split("/")[-2]
    save_path_cls=path_f+fname+'/'
    Path(save_path_cls).mkdir(parents=True, exist_ok=True)
    filename=img.split("/")[-1]
    if img.split('.')[-1] == "gif":
        if os.path.isfile(img):
            dst=processImage(img)  # Convert gif to png file 
            if dst:
                img_o = cv2.imread(dst)
                if img is  None:
                    break
                    os.remove(dst)
                else:
                    print(dst,img) 
        
    else:
        img_o = cv2.imread(img)
        if img_o is not None:
            print(img_o.shape)
            #copyfile(img, save_path_cls+filename) # Copy file to destination
        else:
            print(img)
